# Grabcut

cv2.grabCut (imagem, máscara, retângulo, backgroundModel, foregroundModel, iterationCount [, modo])


### Parâmetros:

**imagem**: imagem de entrada de 3 canais de 8 bits.

**máscara**: máscara de canal único de entrada / saída de 8 bits. A máscara é inicializada pela função quando o modo é definido como **GC_INIT_WITH_RECT**. Seus elementos podem ter um dos seguintes valores:

        1. GC_BGD define pixels de fundo óbvios.
        2. GC_FGD define um pixel óbvio de primeiro plano (objeto).
        3. GC_PR_BGD define um possível pixel de fundo.
        4. GC_PR_FGD define um possível pixel de primeiro plano.
        
**retângulo**: é a região de interesse que contém um objeto segmentado. Os pixels fora da ROI são marcados como plano de fundo. O parâmetro é usado apenas quando mode == GC_INIT_WITH_RECT.

**backgroundModel**: array temporário para o modelo de fundo.

**foregroundModel**: array temporário para o modelo do primeiro plano.

**iterationCount**: Número de iterações que o algoritmo deve fazer antes de retornar o resultado. Observe que o resultado pode ser refinado com outras chamadas com mode == GC_INIT_WITH_MASK ou mode == GC_EVAL.

**modo**: define o modo de operação. Pode ser um dos seguintes:

        1. GC_INIT_WITH_RECT: A função inicializa o estado e a máscara usando o retângulo fornecido. Depois disso, ele executa iterações iterCount do algoritmo.
        2. GC_INIT_WITH_MASK: A função inicializa o estado usando a máscara fornecida. Observe que GC_INIT_WITH_RECT e GC_INIT_WITH_MASK podem ser combinados. Então, todos os pixels fora da ROI são inicializados automaticamente com GC_BGD.
        3. GC_EVAL: O valor significa que o algoritmo deve apenas retomar.

In [2]:
#!Python3

import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('imagem_Exemplo_Grabcut.png')
image_to_show = np.copy(image)

# Estados iniciais do mouse
cropping = False
x_init, y_init, top_left_pt, bottom_right_pt, width, height = 0, 0, 0, 0, 0, 0

def mouse_callback(event, x, y, flags, param):
    global image_to_show, x_init, y_init, top_left_pt, bottom_right_pt, cropping, width, height

    if event == cv2.EVENT_LBUTTONDOWN:
        cropping = True
        x_init, y_init = x, y
        image_to_show = np.copy(image)
        print(f'Ponto inicial em X {x_init}')
        print(f'Ponto inicial em Y {y_init}')

    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping == True:
            image_to_show = np.copy(image)
            cv2.rectangle(image_to_show, (x_init, y_init),
                          (x, y), (0, 255, 0), 1)

    elif event == cv2.EVENT_LBUTTONUP:
        cropping = False
        top_left_pt, bottom_right_pt = x, y
        print(f'Ponto superior {top_left_pt}')
        print(f'Ponto inferior {bottom_right_pt}')

        pontos = [x_init, y_init, top_left_pt, bottom_right_pt]

        width, height = [abs((top_left_pt - x_init)), abs((bottom_right_pt - y_init))]

        print(pontos)
        print(width, height)


cv2.namedWindow('image')
cv2.setMouseCallback('image', mouse_callback)

while True:

    cv2.imshow('image', image_to_show)
    k = cv2.waitKey(1)

    if k == ord('c'):
        if y_init > bottom_right_pt:
            y_init, bottom_right_pt = bottom_right_pt, y_init
        if x_init > top_left_pt:
            x_init, top_left_pt = top_left_pt, x_init

        if bottom_right_pt - y_init > 1 and top_left_pt - x_init > 0:
            image = image[y_init:bottom_right_pt, x_init:top_left_pt]
            mask = np.zeros(image.shape[:2], np.uint8)
            bgd = np.zeros((1, 65), np.float64)
            fgd = np.zeros((1, 65), np.float64)
            rect = (1, 1, width, height)

            cv2.grabCut(image, mask, rect, bgd, fgd, 5, cv2.GC_INIT_WITH_RECT)
            mask2 = np.where((mask == 2)|(mask == 0), 0, 1).astype('uint8') 
            image = image * mask2[:, :, np.newaxis] 
            print(image)
            image_to_show = np.copy(image)

    if k == ord('s'):
        cv2.imwrite('teste.jpg', image_to_show)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()





Ponto inicial em X 967
Ponto inicial em Y 348
Ponto superior 1153
Ponto inferior 528
[967, 348, 1153, 528]
186 180
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
